# 칼로리 개정

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# 데이터셋 로드
food = pd.read_csv('./foodData4_24.csv')

# 결측값 처리(Keywords열의 데이터 중 nan값 공백으로 처리)
# 현재 food 데이터에 일부 깨진데이터(?)가 있어서 결측값 처리해야 정상작동
food['Keywords'] = food['Keywords'].fillna('')

# TF-IDF(Term Frequency-Inverse Document Frequency)
# 텍스트 데이터의 통계적인 가중치를 계산, 처리해주는 기능(자연어 처리)

# TfidfVectorizer : 문서를 벡터 표현으로 바꿔주는 기능
# stop_words='english' => 영어의 일반적인 불용어 삭제(and, is, the, this 등 단어와 상관없는 문자)
tfidf = TfidfVectorizer(stop_words='english')

# food의 Keywords를 단어로 구별
# 단어로 구별된 Keywords를 TF-IDF 행렬로 생성
tfidf_matrix = tfidf.fit_transform(food['Keywords'])

# 사용자 키워드
# 각 사용자의 키워드를 딕셔너리로 조회
# 아래는 예시 데이터

# 사용자 정보 입력
user_info = {
    'Birth date': '26.02.1991',
    'Height': 171,
    'Weight': 74,
    'Name': 'Emmie',
    'Email': 'ecleator2@booking.com',
    'Gender': 'Female'
}

# 데이터 로드
food_data = pd.read_csv('./foodData4_24.csv')

# BMI 계산
height_m = user_info['Height'] / 100
weight_kg = user_info['Weight']
bmi = weight_kg / (height_m ** 2)

# 추정된 칼로리 요구량 계산
if user_info['Gender'] == 'Male':
    bmr = 88.362 + (13.397 * weight_kg) + (4.799 * height_m * 100) - (5.677 * 33)  # 테스트를 위해 임의의 나이 사용
else:
    bmr = 447.593 + (9.247 * weight_kg) + (3.098 * height_m * 100) - (4.330 * 33)

calories_threshold = bmr * 1.2  # 활동 수준을 고려하여 칼로리 요구량을 조정

users = {
    'user1': [['Dessert', 'Pork', 'Grains', 'Oven', 'Christmas', 'Brunch'],[calories_threshold/3]],
    'user2': ['Dessert', 'Berries', 'Fruit', 'Low Protein', 'Brunch'],
    'user3': [['Egg Free', 'Fruit', 'Oven', 'Brunch', 'Berries'], [150]]
}

# 추천 함수 정의
def get_recommendations(user_keywords, tfidf_matrix, food_data):
    # 사용자 키워드를 조회할떼 공백(' ')에 조인하여 하나의 문자열로 처리
    # 사용자 키워드에 대한 TF-IDF 행렬 생성
    user_tfidf_matrix = tfidf.transform([' '.join(user_keywords)])
    
    # 사용자 키워드 행렬과 food 데이터의 키워드 행렬의 코사인 유사도 계산
    cosine_sim = cosine_similarity(user_tfidf_matrix, tfidf_matrix)
   
    # 음식 추천 알고리즘
    # 코사인 유사도 행렬에서 첫 번째 행(사용자 입력과 음식 간의 유사도)을 가져와서
    # 각 음식과의 유사도를 인덱스와 함께 리스트로 변환 => 각 키워드가 가진 유사도를 음식과 전체 조회하여 반환
    sim_scores = list(enumerate(cosine_sim[0]))
   
    # 유사도를 기준으로 리스트를 내림차순으로 정렬
    # key=lambda x: x[1]는 리스트의 각 요소를 유사도에 대해 정렬하기 위한 키 함수
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # 유사도가 가장 높은 음식(sim_scores[0])을 제외하고 반환
    # 가장 높은 음식을 반환하는 이유는 검색기록이 1개인 경우 그 데이터가 가장 높은 유사도를 가지게 되므로
    sim_scores = sim_scores[1:]

    # sim_scores에 저장된 음식들의 인덱스를 저장
    # sim_scores에 저장된 형태 => (인덱스 번호, 유사도 점수)
    food_indices = [idx[0] for idx in sim_scores]

    # sim_scores에 저장된 음식에 대하여 iloc(food_indices)를 통하여 이름 조회
    return food_data.iloc[food_indices][['Name','Calories']]

###
###  2단계. 조건 추가
###

### 사용자가 섭취 가능한 칼로리로 검색조건 추가

# 사용자가 섭취 가능한 일일 칼로리를 추출
user_calories1 = users['user1'][1][0]
user_calories3 = users['user3'][1][0]

print("BMI:", bmi)
print("추정된 칼로리 요구량:", calories_threshold)

# 하나의 사용자에 대한 음식 추천 실행
keywords1 = users['user1']
print("user1의 유사한 음식 추천:")
recommendations1 = get_recommendations(keywords1[0], tfidf_matrix, food)
print(recommendations1)
print()

keywords3 = users['user3']
print("user3의 유사한 음식 추천:")
recommendations3 = get_recommendations(keywords3[0], tfidf_matrix, food)
print(recommendations3)
print()

# 추천된 음식 중에서 사용자가 섭취 가능한 일일 칼로리 이하의 음식만 필터링
print("user1의 필터된 음식 추천:")
filtered_recommendations1 = recommendations1[recommendations1['Calories'] <= user_calories1]
print(filtered_recommendations1)

print("user3의 필터된 음식 추천:")
filtered_recommendations3 = recommendations3[recommendations3['Calories'] <= user_calories3]
print(filtered_recommendations3)
print(filtered_recommendations3[:1])

생일 1


BMI: 25.3069320474676
추정된 칼로리 요구량: 1822.4868
user1의 유사한 음식 추천:
                                                    Name  Calories
142321                          Coffee Chocolate Waffles     459.6
66308       Tina's Overnight Sausage &amp; Egg Casserole     228.8
15217                                            "Tomato     541.5
6498                        Very Soft Rolled Oat Cookies     102.4
14133                       Peggy's Foolproof  Pie Crust     380.4
...                                                  ...       ...
165891                Fun & Delicious Homemade Pop Tarts     576.1
165893                     Spanish Coffee with Tia Maria      84.3
165894                         Meg's Pumpkin Spice Bread     898.2
165895  Roast Prime Rib au Poivre with Mixed Peppercorns    2063.4
165896          Quick & Easy Asian Cucumber Salmon Rolls      16.1

[165896 rows x 2 columns]

user3의 유사한 음식 추천:
                                                    Name  Calories
94520               

In [18]:
recommendations[:2]['Calories']

142321    459.6
66308     228.8
Name: Calories, dtype: float64

In [92]:
filtered_recommendations.shape

(10, 2)

In [84]:
user_calories = users['user1'][1][0]
user_calories

1000

In [49]:
import numpy as np

In [68]:
users['user1'][1]

[400]

In [71]:
user_calories = users['user1'][1][0]

In [72]:
user_calories

400

In [88]:
food['Keywords']

0         Dessert, Low Protein, Low Cholesterol, Healthy...
1         Chicken Thigh & Leg, Chicken, Poultry, Meat, A...
2         Low Protein, Low Cholesterol, Healthy, Summer,...
3         Beans, Vegetable, Low Cholesterol, Weeknight, ...
4         Low Protein, Vegan, Low Cholesterol, Healthy, ...
                                ...                        
165892    Dessert, Cookie & Brownie, < 30 Mins, For Larg...
165893                        < 15 Mins, Easy, From Scratch
165894                              Breads, < 4 Hours, Easy
165895                  High Protein, High In..., < 4 Hours
165896                                      < 15 Mins, Easy
Name: Keywords, Length: 165897, dtype: object

In [127]:
recommend = recommendations3['Name'][:]
recommend_index = food[food['Name'].isin(recommend)].index
keywords_of_recommendations = food['Keywords'].iloc[recommend_index]

In [133]:
print(keywords_of_recommendations)

0         Dessert, Low Protein, Low Cholesterol, Healthy...
1         Chicken Thigh & Leg, Chicken, Poultry, Meat, A...
2         Low Protein, Low Cholesterol, Healthy, Summer,...
3         Beans, Vegetable, Low Cholesterol, Weeknight, ...
4         Low Protein, Vegan, Low Cholesterol, Healthy, ...
                                ...                        
165892    Dessert, Cookie & Brownie, < 30 Mins, For Larg...
165893                        < 15 Mins, Easy, From Scratch
165894                              Breads, < 4 Hours, Easy
165895                  High Protein, High In..., < 4 Hours
165896                                      < 15 Mins, Easy
Name: Keywords, Length: 165896, dtype: object
